<a href="https://colab.research.google.com/github/tthandi/magic_formula_project/blob/main/IncomeStatementLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_vantage_income_statement_full.py — INCOME-STATEMENT

• Quarterly income statements for US stocks (active + delisted).
• Modes:
    full     → all symbols
    gapfill  → symbols missing from BQ
    backfill → symbols already in BQ (row-level backfill; end-of-run de-dup)
• Premium-friendly: CALLS_PER_MIN=75, CONCURRENCY=50, robust async LISTING_STATUS.
• Dynamic schema discovery + auto-evolve (incl. mid-run).
• Safe CSV loads (column-order alignment, mkstemp, WRITE_APPEND to partitions).
• Success-only checkpoint + failure log.
• Per-partition flush locks; end-of-run de-dup on (symbol, fiscalDateEnding).
• Jupyter-friendly CLI (parse_known_args).

Env:
  ALPHAVANTAGE_API_KEY  (required)
  GOOGLE_APPLICATION_CREDENTIALS (optional, path to service account JSON)
"""

# ───────────────────────────── CONFIG ───────────────────────────────
PROJECT   = ""
DATASET   = ""
TABLE     = "income_statement"   # same table name, now quarterly data

import os as _os
API_KEY   = '' # REQUIRED

CALLS_PER_MIN = 75
CONCURRENCY   = 50
FLUSH_ROWS    = 10_000
CHECKFILE     = "av_done_symbols_is.txt"
FAILFILE      = "av_failed_symbols_is.txt"
PART_START, PART_END = 198001, 204001  # yyyymm partition range

# ───────────────────────────── IMPORTS ──────────────────────────────
import io, csv, json, time, asyncio, tempfile, aiohttp, nest_asyncio, argparse
from collections import defaultdict
from typing import Dict, List, Set, Tuple
import pandas as pd
import requests
import tqdm                      # module-style import
from aiolimiter import AsyncLimiter

from google.cloud import bigquery
from google.api_core import exceptions as gex
from google.auth.exceptions import DefaultCredentialsError, RefreshError
from google.oauth2 import service_account

nest_asyncio.apply()

# ───────────────────────── Auth-robust BigQuery client ──────────────
def get_bq_client() -> bigquery.Client:
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        return bigquery.Client(project=PROJECT)
    except Exception:
        pass
    sa_path = _os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if sa_path and _os.path.exists(sa_path):
        creds = service_account.Credentials.from_service_account_file(sa_path)
        return bigquery.Client(project=PROJECT, credentials=creds)
    try:
        return bigquery.Client(project=PROJECT)
    except (DefaultCredentialsError, RefreshError) as e:
        raise RuntimeError(
            "No Google Cloud credentials found. Do one of:\n"
            "  • Colab: from google.colab import auth; auth.authenticate_user()\n"
            "  • Service account: set GOOGLE_APPLICATION_CREDENTIALS to your JSON path\n"
            "  • gcloud: run `gcloud auth application-default login`"
        ) from e

bq = get_bq_client()

# ─────────────────── Alpha Vantage helpers (async) ──────────────────
BASE_URL     = "https://www.alphavantage.co/query"
RATE_LIMITER = AsyncLimiter(CALLS_PER_MIN, 60)

async def av_request(session: aiohttp.ClientSession, params: dict) -> dict | None:
    async with RATE_LIMITER:
        try:
            async with session.get(BASE_URL, params=params, timeout=45) as resp:
                if resp.status != 200:
                    return None
                return await resp.json(content_type=None)
        except (aiohttp.ClientError, asyncio.TimeoutError):
            return None

async def listing(session: aiohttp.ClientSession, state: str, max_retries: int = 6) -> List[str]:
    params = {"function": "LISTING_STATUS", "state": state, "datatype": "csv", "apikey": API_KEY}
    backoff = 2
    for attempt in range(1, max_retries + 1):
        async with RATE_LIMITER:
            try:
                async with session.get(BASE_URL, params=params, timeout=45) as resp:
                    status = resp.status
                    txt = await resp.text()
            except (aiohttp.ClientError, asyncio.TimeoutError):
                tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: network error (try {attempt})")
                await asyncio.sleep(backoff); backoff *= 2
                continue
        if txt.lstrip().startswith("{"):
            try:
                j = json.loads(txt)
            except json.JSONDecodeError:
                j = {}
            msg = j.get("Note") or j.get("Information") or j.get("Error Message") or f"HTTP {status}"
            tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: {msg} (try {attempt})")
            await asyncio.sleep(backoff); backoff = min(backoff * 2, 30)
            continue
        rdr = csv.DictReader(io.StringIO(txt))
        syms = [r["symbol"] for r in rdr if r.get("assetType") == "Stock" and r.get("symbol")]
        if syms:
            return syms
        tqdm.tqdm.write(f"⚠️ LISTING_STATUS {state}: empty CSV (try {attempt})")
        await asyncio.sleep(backoff); backoff *= 2
    tqdm.tqdm.write(f"❗ LISTING_STATUS {state}: giving up after {max_retries} attempts")
    return []

def coerce_numeric(col: pd.Series) -> pd.Series:
    col = col.replace({"None": pd.NA, "null": pd.NA, "": pd.NA})
    return pd.to_numeric(col, errors="coerce")

# ─────────────── fetch QUARTERLY income statement here ──────────────
async def fetch_income_statement(session: aiohttp.ClientSession, sym: str) -> pd.DataFrame | None:
    data = await av_request(
        session,
        {"function": "INCOME_STATEMENT", "symbol": sym, "apikey": API_KEY}
    )
    # use quarterlyReports instead of annualReports
    if not data or not data.get("quarterlyReports"):
        return None

    df = pd.DataFrame(data["quarterlyReports"])
    df["symbol"] = sym

    # SG&A alias normalization
    if "sellingGeneralAndAdministrative" in df.columns and "sellingGeneralAdministrative" not in df.columns:
        df["sellingGeneralAdministrative"] = df["sellingGeneralAndAdministrative"]

    df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"]).dt.date
    df["yyyymm"] = df["fiscalDateEnding"].apply(lambda d: d.year * 100 + d.month).astype("int64")

    non_numeric = {"symbol", "fiscalDateEnding", "reportedCurrency", "yyyymm"}
    numeric_cols = [c for c in df.columns if c not in non_numeric]
    if numeric_cols:
        df[numeric_cols] = df[numeric_cols].apply(coerce_numeric)
    return df

# ───────────────────── BigQuery boot-strapping ─────────────────────
try:
    bq.get_dataset(f"{PROJECT}.{DATASET}")
except gex.NotFound:
    bq.create_dataset(bigquery.Dataset(f"{PROJECT}.{DATASET}"))

table_ref = f"{PROJECT}.{DATASET}.{TABLE}"

SAMPLE_TICKERS = ["AAPL", "IBM", "MSFT", "XOM", "GE", "GOOGL", "AMZN", "BAC"]

def discover_fields(sample_syms: List[str]) -> List[str]:
    """
    Discover all possible fields from both annual and quarterly income statements
    for a small sample of symbols, so our schema covers everything upfront.
    """
    fields: Set[str] = set()
    for sym in sample_syms:
        try:
            resp = requests.get(
                BASE_URL,
                params={"function": "INCOME_STATEMENT", "symbol": sym, "apikey": API_KEY},
                timeout=20
            ).json()
            # include both, in case quarterlyReports has extra keys
            fields |= {k for rpt in resp.get("annualReports", []) for k in rpt}
            fields |= {k for rpt in resp.get("quarterlyReports", []) for k in rpt}
        except Exception:
            continue
        if len(fields) >= 80:
            break
    if "sellingGeneralAndAdministrative" in fields and "sellingGeneralAdministrative" not in fields:
        fields.add("sellingGeneralAdministrative")
    base = ["symbol", "fiscalDateEnding", "reportedCurrency", "yyyymm"]
    return base + sorted(fields - set(base))

FIELDS = discover_fields(SAMPLE_TICKERS)

# ─────────────── FLOAT fix for problematic numeric fields ───────────
_TYPE_OVERRIDES = {
    "symbol": "STRING",
    "fiscalDateEnding": "DATE",
    "reportedCurrency": "STRING",
    "yyyymm": "INT64",
    # IMPORTANT: use FLOAT64 to avoid CSV "1234.0" → INT64 parse errors
    "commonStockSharesOutstanding": "FLOAT64",
}

SCHEMA = [bigquery.SchemaField(col, _TYPE_OVERRIDES.get(col, "FLOAT64")) for col in FIELDS]

def ensure_table() -> Tuple[bool, List[str]]:
    try:
        tbl = bq.get_table(table_ref)
        existing = {f.name for f in tbl.schema}
        missing = [c for c in FIELDS if c not in existing]
        if missing:
            tbl.schema += [bigquery.SchemaField(c, _TYPE_OVERRIDES.get(c, "FLOAT64")) for c in missing]
            bq.update_table(tbl, ["schema"])
            tqdm.tqdm.write(
                f"🆕 Added {len(missing)} new columns → {', '.join(missing[:6])}"
                f"{'…' if len(missing)>6 else ''}"
            )
            return True, missing
        return False, []
    except gex.NotFound:
        tbl = bigquery.Table(table_ref, schema=SCHEMA)
        tbl.range_partitioning = bigquery.RangePartitioning(
            field="yyyymm", range_=bigquery.PartitionRange(PART_START, PART_END, 1)
        )
        tbl.clustering_fields = ["symbol"]
        bq.create_table(tbl)
        tqdm.tqdm.write(f"➡️  created table {table_ref} with {len(SCHEMA)} columns")
        return True, [f.name for f in tbl.schema]

schema_changed_initial, _ = ensure_table()
CSV_COLS = [f.name for f in bq.get_table(table_ref).schema]

# ───────────── Mid-run schema evolve if new columns appear ──────────
_schema_lock = asyncio.Lock()
async def ensure_columns_midrun(new_cols: Set[str]) -> bool:
    if not new_cols:
        return False
    async with _schema_lock:
        tbl = bq.get_table(table_ref)
        existing = {f.name for f in tbl.schema}
        to_add = [c for c in new_cols if c not in existing]
        if not to_add:
            return False
        tbl.schema += [bigquery.SchemaField(c, _TYPE_OVERRIDES.get(c, "FLOAT64")) for c in to_add]
        bq.update_table(tbl, ["schema"])
        global CSV_COLS
        CSV_COLS = [f.name for f in bq.get_table(table_ref).schema]
        tqdm.tqdm.write(f"🆕 (mid-run) added {len(to_add)} cols → {', '.join(to_add)}")
        return True

# ───────────────────── Load helper (partitioned) ───────────────────
def load_partition(df_part: pd.DataFrame, yyyymm: int, retries: int = 4):
    if df_part is None or df_part.empty:
        return
    df_part = df_part.reindex(columns=CSV_COLS)
    fd, tmp = tempfile.mkstemp(suffix=".csv.gz"); _os.close(fd)
    df_part.to_csv(tmp, index=False, compression="gzip", date_format="%Y-%m-%d")
    dest = f"{table_ref}${yyyymm}"
    cfg = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition="WRITE_APPEND",
    )
    backoff = 5
    for attempt in range(1, retries + 1):
        try:
            with open(tmp, "rb") as fh:
                job = bq.load_table_from_file(fh, dest, job_config=cfg, rewind=True)
            tqdm.tqdm.write(f"⬆️  load {dest} rows {len(df_part)} (try {attempt})")
            job.result(timeout=300)
            tqdm.tqdm.write(f"✅ partition {yyyymm} rows {len(df_part)}")
            break
        except (gex.GoogleAPICallError, gex.RetryError, TimeoutError) as e:
            tqdm.tqdm.write(f"⚠️  {dest} {e} — backoff {backoff}s")
            time.sleep(backoff); backoff = min(backoff * 2, 60)
    _os.remove(tmp)

# ───────────────────── Checkpoint / failure helpers ─────────────────
def read_checkpoint() -> Set[str]:
    return set(open(CHECKFILE).read().splitlines()) if _os.path.exists(CHECKFILE) else set()

def add_checkpoint(sym: str):
    with open(CHECKFILE, "a") as f:
        f.write(sym + "\n")

def add_failure(sym: str):
    with open(FAILFILE, "a") as f:
        f.write(sym + "\n")

# ───────────────────────── De-duplication helper ────────────────────
def dedupe_partitions(parts: Set[int]):
    import tqdm as _tqdm
    if not parts:
        _tqdm.tqdm.write("🧹 dedupe: no partitions to process")
        return

    sql = f"""
    FOR part IN (SELECT p FROM UNNEST(@parts) AS p) DO
      CREATE OR REPLACE TEMP TABLE tmp AS
      SELECT * EXCEPT(rn) FROM (
        SELECT t.*,
               ROW_NUMBER() OVER (
                 PARTITION BY symbol, fiscalDateEnding
                 ORDER BY yyyymm DESC
               ) AS rn
        FROM `{table_ref}` AS t
        WHERE yyyymm = part.p      -- ✅ use part.p (struct field)
      )
      WHERE rn = 1;

      DELETE FROM `{table_ref}` WHERE yyyymm = part.p;   -- ✅ use part.p
      INSERT INTO `{table_ref}` SELECT * FROM tmp;
    END FOR;
    """

    job = bq.query(
        sql,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ArrayQueryParameter("parts", "INT64", sorted(parts))
            ]
        ),
    )
    job.result()
    _tqdm.tqdm.write(f"🧹 dedupe: processed {len(parts)} partition(s)")

# ───────────────────────── Main async entrypoint ────────────────────
async def run(mode: str):
    if not API_KEY:
        raise SystemExit("❌  Set ALPHAVANTAGE_API_KEY and retry.")

    sem                         = asyncio.Semaphore(CONCURRENCY)
    pending: Dict[int, List[pd.DataFrame]] = defaultdict(list)
    pending_rows: Dict[int, int]           = defaultdict(int)
    part_locks: Dict[int, asyncio.Lock]    = {}
    touched_parts: Set[int]                = set()
    schema_changed_flag: bool              = schema_changed_initial

    async def maybe_flush(part: int):
        lock = part_locks.setdefault(part, asyncio.Lock())
        async with lock:
            if pending_rows[part] >= FLUSH_ROWS:
                batch = pd.concat(pending[part]); pending[part].clear(); pending_rows[part] = 0
                await asyncio.to_thread(load_partition, batch, part)

    connector = aiohttp.TCPConnector(limit=CONCURRENCY)
    async with aiohttp.ClientSession(
        connector=connector,
        timeout=aiohttp.ClientTimeout(total=90)
    ) as session:
        active, delisted = await asyncio.gather(
            listing(session, "active"),
            listing(session, "delisted")
        )
        tqdm.tqdm.write(
            f"ℹ️  LISTING_STATUS → active: {len(active):,}, delisted: {len(delisted):,}"
        )
        universe = sorted(set(active + delisted))
        if not universe:
            tqdm.tqdm.write("❗ LISTING_STATUS returned 0; aborting (premium key expected to work).")
            return

        # Build mode-specific TODO
        existing = {
            row.symbol
            for row in bq.query(f"SELECT DISTINCT symbol FROM `{table_ref}`").result()
        }
        if mode == "auto":
            mode = "full" if len(existing) == 0 else "gapfill"

        if mode == "full":
            todo = universe
            todo_desc = "FULL crawl (quarterly)"
        elif mode == "gapfill":
            todo = [s for s in universe if s not in existing]
            todo_desc = "GAP-FILL (missing symbols only, quarterly)"
        elif mode == "backfill":
            todo = [s for s in universe if s in existing]
            todo_desc = "BACKFILL (existing symbols only, quarterly)"
        else:
            raise SystemExit(f"Unknown mode: {mode}")

        tqdm.tqdm.write(
            f"Mode: {todo_desc} | universe {len(universe):,} | "
            f"already in BQ {len(existing):,} | to fetch {len(todo):,}"
        )

        pbar = tqdm.tqdm(total=len(todo), desc="Symbols processed", dynamic_ncols=True)

        async def worker(sym: str):
            nonlocal schema_changed_flag
            async with sem:
                df = await fetch_income_statement(session, sym)
                if df is None or df.empty:
                    add_failure(sym)
                    pbar.update(1)
                    return

                new_cols = set(df.columns) - set(CSV_COLS)
                if new_cols:
                    changed = await ensure_columns_midrun(new_cols)
                    schema_changed_flag = schema_changed_flag or changed

                for part, grp in df.groupby("yyyymm"):
                    pending[part].append(grp)
                    pending_rows[part] += len(grp)
                    touched_parts.add(int(part))
                    await maybe_flush(part)

                add_checkpoint(sym)  # success-only
                pbar.update(1)

        tasks = [asyncio.create_task(worker(s)) for s in todo]
        await asyncio.gather(*tasks)
        pbar.close()

    # flush any remaining
    for part, dfs in pending.items():
        if dfs:
            await asyncio.to_thread(load_partition, pd.concat(dfs), part)

    # de-dup quarterly rows per (symbol, fiscalDateEnding)
    await asyncio.to_thread(dedupe_partitions, touched_parts)

# ────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--mode",
        choices=["auto", "full", "gapfill", "backfill"],
        default="auto",
        help="auto=full if empty else gap-fill; backfill=existing symbols only",
    )
    args, _unknown = parser.parse_known_args()
    asyncio.run(run(args.mode))
